# 2: Extracting Global Feature Weighting:

### Pedagogical 
1. Perturbation Method 
2. Sensitivity (Im et al. 2007) 

### Decompositional
1. Garson's Algorithm. 
2. Connection Weights. 

In [1]:
import pandas as pd
import numpy as np
import pickle

from keras.models import load_model

from copy import deepcopy

from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error

Using TensorFlow backend.


## Load Models and Data:

In [2]:
df = pd.read_csv("processed_df.csv")

In [3]:
knn_clf = pickle.load(open("k-nn_model.sav", 'rb'))
model = load_model("NN.h5")

In [4]:
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

In [5]:
# To keep track of what the numpy matrix columns are
feature_names = df.columns

## Pedagogical
### Sensitivity (Im et al. 2007)

$$
S_i = \frac{(\sum_{L} \frac{|P^0 - P^i|}{P^0})}{n}
$$

Where $P^0$ is the normal prediction value for each training instance after training. <br>
$P^i$ is the modified prediction when input node is removed. <br>
$L$ is the set of training data. <br>
$n$ is the number of training data instances.

In [6]:
# Iterate though each feature, remove it from the input data and evalutate above forumla for get weights

sensitivity_weights = dict()

for i in range(len(feature_names)):
    feature = feature_names[i]
    temp_df = deepcopy(X_test)
    
    # Remove Input
    temp_df = temp_df.T
    temp_df[i] = 0
    temp_df = temp_df.T
    
    numerator = 0
    denomonator = len(X_test)
    
    for j in range(len(X_test)):
        p0 = model.predict_proba(np.array([X_test[j]]))[0][0]
        pi = model.predict_proba(np.array([temp_df[j]]))[0][0]
        numerator += (abs(p0 - pi) / p0)
        
    si = numerator / denomonator
    sensitivity_weights[feature] = si
    
sensitivity_weights

{'LIMIT_BAL': 0.36555495648113234,
 'AGE': 0.09980291325518738,
 'PAY_0': 4.088079244620865,
 'PAY_2': 0.5472792245711559,
 'PAY_3': 0.3929872988997995,
 'PAY_4': 0.2914931618846628,
 'PAY_5': 0.23134937200139513,
 'PAY_6': 0.21140192810107375,
 'BILL_AMT1': 0.19738257473548584,
 'BILL_AMT2': 0.21833864640359632,
 'BILL_AMT3': 0.08671430777171084,
 'BILL_AMT4': 0.13725055436902614,
 'BILL_AMT5': 0.12588156242955847,
 'BILL_AMT6': 0.11677851517353671,
 'PAY_AMT1': 0.14455251931410687,
 'PAY_AMT2': 0.14977028905386153,
 'PAY_AMT3': 0.14997392137235874,
 'PAY_AMT4': 0.11766988622622163,
 'PAY_AMT5': 0.09332286575252602,
 'PAY_AMT6': 0.18032908941836406,
 'SEX_1': 0.14792031993609636,
 'SEX_2': 0.1339676630050999,
 'EDUCATION_0': 0.2813087149665868,
 'EDUCATION_1': 0.30166202735915915,
 'EDUCATION_2': 0.2799833689026212,
 'EDUCATION_3': 0.18653849067494108,
 'EDUCATION_4': 0.21685637820624834,
 'EDUCATION_5': 0.13520752381981704,
 'EDUCATION_6': 0.2569881056167396,
 'MARRIAGE_0': 0.1331175

In [7]:
np.save('sensitivity_weights.npy', sensitivity_weights) 

### Perturbation Method

$ x = x +- \sigma $

Where $\sigma$ is a perturbation of 0-20% of the input $x$. There was a recent paper by Runbo et al. http://www.jcomputers.us/vol6/jcp0607-16.pdf showing the optimal perturbation range to be 20%.

Theodor et al. https://www.dcl.hpi.uni-potsdam.de/papers/papers/heinze-feature-salience.pdf also showed perturbation to be the most promising method of retrieving feature ranking from a NN when compared to their own algorithm and connection weights.

##### Our Proposed Weighting Formulation
$ W_i = \frac{\sum_{j=1}^{n}\delta (\sigma, L_j)}{2n} $ 

where $W_i$ represents the global weight of feature $i$, $\sigma$ the perturbation range, $n$ the number of training set instances, $L$ the training data, and $\delta$ a function returning the absolute summed change in two predictions with a positive and negative perturbation respectively to the feature $i$ in instance $L_j$.


### First +20%

In [8]:
perturb_up = dict()
perturbation_range = 0.4  # +20% Since data is normalised between -1 => +1

# Prepare dicitonary for feature weights
for feature in feature_names:
    perturb_up[feature] = 0

# Get original prediction before perturbation
for i in range(len(X_test)):
    
    model_output = model.predict_proba(np.array([X_test[i]]))[0][0]
    
    for j in range(len(feature_names)):
        perturb_instance = deepcopy(X_test[i])
        perturb_instance[j] += perturbation_range
        
        perturb_output = model.predict_proba(np.array([perturb_instance]))[0][0]
        change = abs(model_output - perturb_output)
        perturb_up[feature_names[j]] += change
        
perturb_up

{'LIMIT_BAL': 291.8165729548782,
 'AGE': 89.83324689976871,
 'PAY_0': 2070.7176672723144,
 'PAY_2': 407.48857374303043,
 'PAY_3': 259.95386372320354,
 'PAY_4': 221.7571383677423,
 'PAY_5': 169.57525571249425,
 'PAY_6': 165.49539516493678,
 'BILL_AMT1': 170.75871409662068,
 'BILL_AMT2': 135.77834891341627,
 'BILL_AMT3': 64.19566868431866,
 'BILL_AMT4': 108.24045379646122,
 'BILL_AMT5': 87.28869246691465,
 'BILL_AMT6': 114.4857925157994,
 'PAY_AMT1': 64.2204763237387,
 'PAY_AMT2': 88.2576067801565,
 'PAY_AMT3': 105.54655730910599,
 'PAY_AMT4': 57.70376958139241,
 'PAY_AMT5': 59.57812053710222,
 'PAY_AMT6': 136.43784589506686,
 'SEX_1': 65.49494473077357,
 'SEX_2': 52.31237966194749,
 'EDUCATION_0': 114.07024168223143,
 'EDUCATION_1': 138.53161038830876,
 'EDUCATION_2': 141.79146474599838,
 'EDUCATION_3': 87.04637013934553,
 'EDUCATION_4': 119.96509080007672,
 'EDUCATION_5': 61.282263837754726,
 'EDUCATION_6': 97.12521517835557,
 'MARRIAGE_0': 73.44656120613217,
 'MARRIAGE_1': 113.1766255

### Now -20%

In [9]:
perturb_down = dict()
perturbation_range = -0.4  # +20% Since data is normalised between -1 => +1

# Prepare dicitonary for feature weights
for feature in feature_names:
    perturb_down[feature] = 0

# Get original prediction before perturbation
for i in range(len(X_test)):
    
    model_output = model.predict_proba(np.array([X_test[i]]))[0][0]
    
    for j in range(len(feature_names)):
        perturb_instance = deepcopy(X_test[i])
        perturb_instance[j] += perturbation_range
        
        perturb_output = model.predict_proba(np.array([perturb_instance]))[0][0]
        change = abs(model_output - perturb_output)
        perturb_down[feature_names[j]] += change
        
perturb_down

{'LIMIT_BAL': 357.02522126957774,
 'AGE': 70.44857372716069,
 'PAY_0': 438.43412312306464,
 'PAY_2': 223.5622294358909,
 'PAY_3': 160.6538814753294,
 'PAY_4': 126.96168173104525,
 'PAY_5': 121.96668748185039,
 'PAY_6': 138.7631885614246,
 'BILL_AMT1': 153.6645695734769,
 'BILL_AMT2': 150.03131825849414,
 'BILL_AMT3': 66.71190563030541,
 'BILL_AMT4': 148.78792072087526,
 'BILL_AMT5': 100.61124344542623,
 'BILL_AMT6': 144.93596340715885,
 'PAY_AMT1': 58.53202279098332,
 'PAY_AMT2': 104.12175029329956,
 'PAY_AMT3': 118.68184467405081,
 'PAY_AMT4': 64.60520307347178,
 'PAY_AMT5': 77.90462560392916,
 'PAY_AMT6': 176.14139191433787,
 'SEX_1': 61.64236407727003,
 'SEX_2': 53.556175826117396,
 'EDUCATION_0': 98.38287956826389,
 'EDUCATION_1': 112.0876208646223,
 'EDUCATION_2': 119.23504830244929,
 'EDUCATION_3': 88.36225634627044,
 'EDUCATION_4': 136.02513209730387,
 'EDUCATION_5': 56.821483893319964,
 'EDUCATION_6': 74.72056831791997,
 'MARRIAGE_0': 72.25396698713303,
 'MARRIAGE_1': 107.74383

In [10]:
# Average Scores going up and down
perturb_weights = dict()

for key, value in perturb_up.items():
    # Divide the values by the number of training samples to normalise the results
    # Also divide by two to normalise the up and down sampling
    perturb_weights[key] = ( perturb_up[key] + perturb_down[key] ) / ( 2 * len(X_train) )
    
perturb_weights

{'LIMIT_BAL': 0.013517537379676165,
 'AGE': 0.0033392045963943624,
 'PAY_0': 0.052273995633237064,
 'PAY_2': 0.013146891732894194,
 'PAY_3': 0.008762661358302769,
 'PAY_4': 0.0072649754187247405,
 'PAY_5': 0.0060737904832155135,
 'PAY_6': 0.006338720494299196,
 'BILL_AMT1': 0.0067588184097937,
 'BILL_AMT2': 0.0059543680660814665,
 'BILL_AMT3': 0.0027272411315546682,
 'BILL_AMT4': 0.00535475780244451,
 'BILL_AMT5': 0.003914581998173768,
 'BILL_AMT6': 0.00540461991506163,
 'PAY_AMT1': 0.0025573437315567086,
 'PAY_AMT2': 0.004007903272363667,
 'PAY_AMT3': 0.004671425041315767,
 'PAY_AMT4': 0.002548103596976337,
 'PAY_AMT5': 0.002864223877938154,
 'PAY_AMT6': 0.006512067454362599,
 'SEX_1': 0.0026486939335009082,
 'SEX_2': 0.0022055949060013516,
 'EDUCATION_0': 0.004426106692718652,
 'EDUCATION_1': 0.0052212339844360635,
 'EDUCATION_2': 0.005438052355175993,
 'EDUCATION_3': 0.003654346385116999,
 'EDUCATION_4': 0.005333129643695429,
 'EDUCATION_5': 0.0024604947443973894,
 'EDUCATION_6': 0.

In [11]:
np.save('perturb_weights.npy', perturb_weights) 

## Decompositional

### Garson's Algorithm
Shown by Olden et al. 2002 ftp://gis.msl.mt.gov/Maxell/Models/Predictive_Modeling_for_DSS_Lincoln_NE_121510/Modeling_Literature/Olden_ANN's.pdf

#### Step 1: Calculate the product of each input weight with the hidden layer weight

In [12]:
first_layer_weights = model.layers[0].get_weights()[0]
second_layer_weights = model.layers[2].get_weights()[0]

In [13]:
# (hidden_nodes, inputs)
W1 = first_layer_weights.T
W2 = second_layer_weights.T
Qih_dict = dict()

for h in range(len(W1)):
    for i in range(len(W1[0])):
        Qih_dict["H_" + str(h) + "I_" + str(i)] = W1[h][i] * W2[0][h]

#### Step 2: Relative Contribution
For each input neuron to the outgoing signal of each hidden neuron.

In [14]:
Qih_dict2 = dict()

for h in range(len(W1)):
    for i in range(len(W1[0])):
        
        numerator = abs(Qih_dict["H_" + str(h) + "I_" + str(i)])
        denomonator = 0
        
        for j in range(len(W1[0])):
            denomonator += abs(Qih_dict["H_" + str(h) + "I_" + str(j)])
            
        Qih_dict2["H_" + str(h) + "I_" + str(i)] = numerator / denomonator

In [15]:
# Also sum input neuron contributions
for i in range(len(W1[0])):
    total = 0
    for h in range(len(W1)):
        total += Qih_dict2["H_" + str(h) + "I_" + str(i)]
    Qih_dict2["Sum_input_" + str(i)] = total

#### Step 3: Relative Contribution of each input variable

In [16]:
garsons_weights = dict()

for i in range(len(W1[0])):
    total = 0
    for j in range(len(W1[0])):
        total += Qih_dict2["Sum_input_" + str(j)]
    garsons_weights["Input_" + str(i)] = Qih_dict2["Sum_input_" + str(i)] / total

In [17]:
for i in range(len(feature_names)):
    feature = feature_names[i]
    garsons_weights[feature] = garsons_weights.pop("Input_" + str(i)) 

In [18]:
garsons_weights

{'LIMIT_BAL': 0.04395260965971895,
 'AGE': 0.026944057167293797,
 'PAY_0': 0.08343179461817793,
 'PAY_2': 0.042876711847940556,
 'PAY_3': 0.0359065599004902,
 'PAY_4': 0.03532229472444586,
 'PAY_5': 0.04251643962484055,
 'PAY_6': 0.042533212769370256,
 'BILL_AMT1': 0.029527755927775734,
 'BILL_AMT2': 0.031253182778824405,
 'BILL_AMT3': 0.02446191531544625,
 'BILL_AMT4': 0.032938582451137906,
 'BILL_AMT5': 0.02246390869742399,
 'BILL_AMT6': 0.03718398346177904,
 'PAY_AMT1': 0.022172380196296755,
 'PAY_AMT2': 0.0269092586235034,
 'PAY_AMT3': 0.024459912479915906,
 'PAY_AMT4': 0.029573208742359886,
 'PAY_AMT5': 0.018185671530820045,
 'PAY_AMT6': 0.019636264385791632,
 'SEX_1': 0.021906556147777038,
 'SEX_2': 0.023203353136078075,
 'EDUCATION_0': 0.024656752702645154,
 'EDUCATION_1': 0.030158694252737376,
 'EDUCATION_2': 0.025891523554172294,
 'EDUCATION_3': 0.027243084603266812,
 'EDUCATION_4': 0.02914003814046445,
 'EDUCATION_5': 0.02696126882212165,
 'EDUCATION_6': 0.023007187624478474,

In [19]:
np.save('garsons_weights.npy', garsons_weights) 

### Connection Weights Algorithm
Introducted by Olden et al. ftp://gis.msl.mt.gov/Maxell/Models/Predictive_Modeling_for_DSS_Lincoln_NE_121510/Modeling_Literature/Olden_ANN's.pdf

$ R_{ij} = \sum_{k=1}^{H}W_{ik}.W_{kj} $

Where $k$ is a hidden neuron, $i$ is the input neuron and $j$ is the output neuron.

In [20]:
connection_weights = dict()
W1 = first_layer_weights
W2 = second_layer_weights

for i in range(len(W1)):
    total = 0
    for h in range(len(W1[0])):
        total += W1[i][h] * W2[h]
    connection_weights["Input_" + str(i)] = total[0]

In [21]:
for i in range(len(feature_names)):
    feature = feature_names[i]
    connection_weights[feature] = connection_weights.pop("Input_" + str(i)) 

In [22]:
connection_weights

{'LIMIT_BAL': -1.300265,
 'AGE': 0.73999935,
 'PAY_0': 3.2776766,
 'PAY_2': -3.7349591,
 'PAY_3': -0.12929907,
 'PAY_4': -0.43788692,
 'PAY_5': 0.70825404,
 'PAY_6': 0.4893832,
 'BILL_AMT1': 1.1666768,
 'BILL_AMT2': 0.39782614,
 'BILL_AMT3': 0.02625596,
 'BILL_AMT4': -0.3857166,
 'BILL_AMT5': 0.6940872,
 'BILL_AMT6': -0.9198411,
 'PAY_AMT1': -0.023594515,
 'PAY_AMT2': 0.14318544,
 'PAY_AMT3': -1.1518391,
 'PAY_AMT4': 0.44849443,
 'PAY_AMT5': -0.009882068,
 'PAY_AMT6': -0.8673515,
 'SEX_1': 0.105939336,
 'SEX_2': -0.010222288,
 'EDUCATION_0': 0.3701133,
 'EDUCATION_1': 0.7569208,
 'EDUCATION_2': 0.88181597,
 'EDUCATION_3': 0.55383164,
 'EDUCATION_4': -0.5475441,
 'EDUCATION_5': 0.5473438,
 'EDUCATION_6': 0.7352612,
 'MARRIAGE_0': -0.7086076,
 'MARRIAGE_1': 1.097823,
 'MARRIAGE_2': 0.85597163,
 'MARRIAGE_3': 0.78602546}

In [23]:
np.save('connection_weights.npy', connection_weights) 